# Printout Data | Item Count | Pipe Separated File

In [1]:
import pandas as pd
import re
from pathlib import Path
import os

In [2]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/outputs/")

In [3]:
from ipyfilechooser import FileChooser

In [4]:
fc = FileChooser("C:/Playground/bhaiya-orders/inputs/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/inputs/', filename='', show_hidden='False')

In [5]:
FILE = Path(fc.selected)

In [6]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: df = pd.read_excel(FILE)
except OSError:
    print("Error occured, invalid file!")

In [7]:
df.fillna('', inplace=True)

In [8]:
df.head(2)

,Submission Date,Order Number#,Token,First Name,Last Name,Select Drop Point and Building,Any other building?,Select Wing,Flat No,Phone Number,Email,Select Delivery Date,Your Order: Products,No Label,Any pending unadjusted credit from previous orders?
0,2020-07-22 15:05:42,BHOR-0947,1,Neha,Agrawal,Aeon/Zeon - Aeon,,A Wing,A1604,9930851066,neha31verma@gmail.com,Friday 24th July,"Baby Potato 500 Gms (Amount: 20.00 INR, Specia...",Accepted\n\n1. All issues and complaints regar...,
1,2020-07-21 12:59:12,BHOR-0908,2,Madhavi,Ravuri,Aeon/Zeon - Aeon,,A Wing,803,9820157581,ravurimadhavi@ymail.com,Friday 24th July,"Potato 1 Kg (Amount: 40.00 INR, Special Quanti...",Accepted\n\n1. All issues and complaints regar...,


In [9]:
# df['Token'] = list(range(len(df)))

In [10]:
for col in df.columns:
    if "Your Order" in col:
        your_order = col
        
    if "token" in col.lower():
        token = col
        
    if "flat" in col.lower():
        flat = col
        
    if "order" in col.lower() and "num" in col.lower():
        order_number = col

In [11]:
df['name'] = df['First Name'] + ' ' + df['Last Name'] + '_' + df[token].astype(str)

In [12]:
df[your_order][0]

'Baby Potato 500 Gms (Amount: 20.00 INR, Special Quantity: 1)\nOnion Red 1 Kg (Amount: 18.00 INR, Special Quantity: 1)\nBitter Gourd 300 Gms (Amount: 17.00 INR, Special Quantity: 1)\nButton Mushroom 200 gms Pack (Amount: 60.00 INR, Special Quantity: 1)\nTomato 1 Kg (Amount: 52.00 INR, Special Quantity: 1)\nBroccoli  250 gms (Amount: 50.00 INR, Special Quantity: 1)\nLemons pack of 5  (Amount: 15.00 INR, Special Quantity: 1)\nCoriander Leaves per bunch 250 - 300 gms (Amount: 60.00 INR, Special Quantity: 2)\nPalak/ Spinach Leaves per bunch 400 gms - 500 gms (Amount: 30.00 INR, Special Quantity: 1)\nMethi Leaves per bunch 450 gms - 500 gms (Amount: 40.00 INR, Special Quantity: 1)\nSpring Onions per bunch (Amount: 22.00 INR, Quantity: 1)\nBasil Leaves  200 gms (Amount: 40.00 INR, Special Quantity: 1)\nSubtotal: \nTax: \n\nTotal: 424.00'

In [13]:
k = []
for order in df[your_order]:
    split = order.split(')') # last item is always total cost
    split = [s.replace('\n', '') for s in split] # remove newlines if present
    k.append(split)

In [14]:
save_pth = OUTPUT_DIR/FILE.stem
os.makedirs(save_pth, exist_ok=True)

## Non-Fruits (Item Count)

In [15]:
def get_qty(txt):
    
    """
        Returns the Special Quantity key for a given order. 
            Sample input string: '...no. of 500 gm packets required: 1)'
            Returns: 1
    """
    
    match = re.search(": ([0-9]+?)\)", txt)
    if match:
        return int(match.group(1))

In [16]:
val = {}
total = {}
for name, order in zip(df.name, k):
    val[name] = 0
    for i in order:
        sp1 = i.split("(")
        if len(sp1) < 2:
            total[name] = sp1[0].split()[-1] # total cost
            continue
        
        sp1[1] = sp1[1] + ')'
        val[name] += get_qty(sp1[1])
        

In [17]:
new_df = pd.DataFrame()

In [18]:
new_df['token_no'] = df[token]

In [19]:
new_df['order_no'] = df[order_number]

In [20]:
new_df['name'] = df['name'].apply(lambda x: x.split('_')[0])

In [21]:
new_df['flat_no'] = df[flat]

In [22]:
# # For Thane
# new_df['wing'] = df['Select Wing']

In [23]:
def func(x):
    if '-' in x:
        return x.split('-')[1]
    return x

new_df['building_name'] = df['Select Drop Point and Building'].apply(func)

In [24]:
new_df['building_name'] = new_df['building_name'] + ', ' + df['Select Wing']

In [25]:
new_df['phone_no'] = df['Phone Number']

In [26]:
new_df['email'] = df['Email']

In [27]:
# # # if Thane
# new_df['drop_point'] = df['Enter Building Name']

In [28]:
new_df['drop_point'] = df['Select Drop Point and Building'].apply(lambda x: x.split('-')[0])

In [29]:
new_df['item_count'] = df.name.apply(lambda x: val[x])

In [30]:
new_df['total_amt'] = df.name.apply(lambda x: total[x])

In [31]:
new_df.head()

,token_no,order_no,name,flat_no,building_name,phone_no,email,drop_point,item_count,total_amt
0,1,BHOR-0947,Neha Agrawal,A1604,"Aeon, A Wing",9930851066,neha31verma@gmail.com,Aeon/Zeon,13,424.00
1,2,BHOR-0908,Madhavi Ravuri,803,"Aeon, A Wing",9820157581,ravurimadhavi@ymail.com,Aeon/Zeon,35,"1,565.00"
2,3,BHOR-0891,Ronak Surana,1304,"Aeon, A Wing",9769656431,ronaksurana29@hotmail.com,Aeon/Zeon,13,646.00
3,4,BHOR-0932,Pooja Verma,1201,"Zeon, A Wing",9920411189,poojalverma1208@gmail.com,Aeon/Zeon,26,896.00
4,5,BHOR-0884,SAUMYA SARAOGI,204,"Zeon, A Wing",9833992919,SAUMYASARAOGI@GMAIL.COM,Aeon/Zeon,18,616.00


In [32]:
fname = save_pth/('printout-' + str(FILE.stem) + '.csv')

In [33]:
new_df.to_csv(fname, index=False)

In [38]:
# new_df[new_df['name'] == 'Suparna Bhandari']

## Fruits (Pipe Separated Orders)

In [15]:
def get_item_name(order):
    '''
    Return the Item Name, Final Weight, Num Packets, Unit
    '''
    
    stopwords = ['per', 'gms', 'gm', 'kgs', 'kg', 'pack', 'each', '-']
    
    # parse the key, stop when you encounter a number, unit, keyword like 'per', 'pack'
    item = order.split('(')[0].strip()
    item = re.sub('[0-9]+', ' ', item)
    item = item.split()
#     print(item)
    final_item = ''
    for token in item:
        if token.lower() in stopwords:
            break
        final_item += ' ' + token
    
    return final_item.strip()


def get_qty(txt):
    
    """
        Returns the Special Quantity key for a given order. 
            Sample input string: '...no. of 500 gm packets required: 1)'
            Returns: 1
    """
    
    if txt[-1] != ')':  txt = txt + ')'
    
    match = re.search(": ([0-9]+?)\)", txt)

    # temporary fix for VNR Guava
    if "VNR Guava Per Kg (" in txt: match = re.search("\(([0-9]+?)", txt)
    
    if match:
        return int(match.group(1))
    
    
def extract_num(txt):
    
    """
        Searches a string and returns the last number found, if present. 
        Otherwise return 1.
        Use case: For an item key such as 'Tondli 300 gms ', return 300 (to calculate total qty required)
    """
    
    match = re.findall("([0-9]+)", txt)
    if match:
        return int(match[-1])
    else:
        return 1
    
    
    
def get_unit(txt):
    """
        Gets unit from an item string.
        Assumption: standard occurence of units all across the board. 
    """
    txt = txt.lower()
    if txt.find(' gm') != -1: unit = 'gms'
    elif (txt.find(' pc') != -1) or (txt.find(' pack') != -1): unit = 'pcs'
    elif txt.find(' bunch') != -1: unit = 'bunches'
    else: unit = 'kgs'
    return unit

In [16]:
def parse_order(order):
    '''
    Return the Item Name, Final Weight, Num Packets, Unit
    '''
    if len(order.split('(')) < 2: return None
    
    item_name = get_item_name(order)
    num_packets = get_qty(order)
    unit = get_unit(order)
    qty_per_pack = extract_num(order.split('(')[0])
    final_weight = qty_per_pack * num_packets

    return {"item_name": item_name,
            "num_packets": num_packets,
            "unit": unit,
            "final_weight": final_weight}

In [17]:
val = {}
total = {}
for name, order in zip(df.name, k):
    val[name] = ''
    for i in order:
        sp1 = i.split("(")
        if len(sp1) < 2:
            total[name] = sp1[0].split()[-1] # total cost
            continue
        
        sp1[1] = sp1[1] + ')'
        val[name] += get_item_name(sp1[0]) + ':' + str(get_qty(sp1[1])) + ' | '

In [18]:
new_df = pd.DataFrame()

In [19]:
new_df['token_no'] = df[token]
new_df['order_no'] = df[order_number]

new_df['name'] = df['name'].apply(lambda x: x.split('_')[0])

new_df['flat_no'] = df[flat]

def func(x):
    if '-' in x:
        return x.split('-')[1]
    return x

# # THANE
# new_df['drop_point'] = df['Enter Building Name']

new_df['building_name'] = df['Select Drop Point and Building'].apply(func)
new_df['building_name'] = new_df['building_name'] + ', ' + df['Select Wing']

new_df['phone_no'] = df['Phone Number']

new_df['email'] = df['Email']

# THANE, COMMENT OUT
new_df['drop_point'] = df['Select Drop Point and Building'].apply(lambda x: x.split('-')[0])

new_df['items'] = df.name.apply(lambda x: val[x][:-3])

new_df['total_amt'] = df.name.apply(lambda x: total[x])

In [20]:
new_df.head(2)

,token_no,order_no,name,flat_no,building_name,phone_no,email,drop_point,items,total_amt
0,1,BHORS-0557,Rashmita Sundaram,1202,"Aeon, A",9004913659,rashmita.sundaram@gmail.com,Aeon/Zeon,Sitaphal/Custard Apple:1 | Elaichi Banana One ...,210.00
1,2,BHORS-0548,Heena Parekh,1101,"Aeon, A",9892404976,heenaparekh75@gmail.com,Aeon/Zeon,Muskmelon:2 | Sitaphal/Custard Apple:1,240.00


In [21]:
new_df.total_amt.apply(lambda x: x.replace(',', '')).astype(float).sum()

35778.0

In [22]:
fname = save_pth/('printout-' + str(FILE.stem) + '.csv')

In [23]:
new_df.to_csv(fname, index=False)

## Finished!